In [1]:
#!/usr/bin/env python3
import pandas as pd
#!pip install sqlalchemy
from sqlalchemy import create_engine
#!pip install psycopg2-binary
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='sqlalchemy')
# Connexion à la base de données MySQL
USER = 'root'
PASSWORD = 'rootroot' 
HOST = '127.0.0.1'
PORT = '3306'
DATABASE = 'credit_risk_db'

try:
    # Création de l'URL de connexion
    engine_url = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
    engine = create_engine(engine_url)
    print("Connexion réussie à la base de données MySQL.")
    # Exemple de requête pour vérifier la connexion
    query = "SELECT * FROM loans"
    # Chargement des données dans un DataFrame pandas
    df = pd.read_sql(query, con=engine)
    # Affichage des premières lignes du DataFrame
    display(df.head())
    
except Exception as e:
    print(f"Erreur de connexion à la base de données MySQL: {e}")
    

Connexion réussie à la base de données MySQL.


,id,loan_amount,interest_rate,term_months,start_date,end_date,borrower_age,borrower_income,credit_score,defaulted,is_simulated_anomaly
0,1,16795,0.1441,36,2024-02-01,2027-01-16,25,130268,320,1,0
1,2,38194,0.0851,60,2024-08-03,2029-07-08,53,80263,430,1,0
2,3,1769,0.1166,24,2025-10-01,2027-09-21,38,103104,759,0,0
3,4,72932,0.0818,12,2025-08-31,2026-08-26,44,45658,810,1,0
4,5,3747,0.0740,36,2023-09-03,2026-08-18,61,76886,543,0,0


In [2]:
# Préparation des données pour le scoring
df_scoring = df[(df['loan_amount'] > 0) & (df['interest_rate'] < 0.5) & (df['borrower_income'] < 1000000)].copy()
# Sélection des caractéristiques
features_scoring = ['credit_score', 'loan_amount', 'interest_rate', 'borrower_income', 'term_months', 'borrower_age']

target_scoring = 'defaulted'
# Séparation des caractéristiques et de la cible
X_scoring = df_scoring[features_scoring]
Y_scoring = df_scoring[target_scoring]

# Préparation des données pour la détection d'anomalies
features_anomalie = ['credit_score', 'loan_amount', 'interest_rate', 'borrower_income']

X_anomalie = df[features_anomalie]

# Affichage des dimensions des ensembles de données
print(f"Dimensions des données pour le scoring : {X_scoring.shape}")
print(f"Dimensions des données pour la détection d'anomalies : {X_anomalie.shape}")



Dimensions des données pour le scoring : (72274, 6)
Dimensions des données pour la détection d'anomalies : (75000, 4)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scoring, Y_scoring, test_size=0.2, random_state=42, stratify=Y_scoring)

# Affichage des dimensions des ensembles
print(f"Set d'entraînement : {X_train.shape[0]} lignes")
print(f"Set de test : {X_test.shape[0]} lignes")

# Initialisation et entraînement du modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Entraînement du modèle
rf_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Évaluation du modèle
print("Rapport de classification :")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Set d'entraînement : 57819 lignes
Set de test : 14455 lignes
Rapport de classification :
              precision    recall  f1-score   support

           0       0.78      0.80      0.79      9134
           1       0.64      0.60      0.62      5321

    accuracy                           0.73     14455
   macro avg       0.71      0.70      0.71     14455
weighted avg       0.73      0.73      0.73     14455

Accuracy: 0.7294


In [6]:
import joblib
import os
# Création du répertoire 'models' s'il n'existe pas
os.makedirs('api', exist_ok=True)
# Sauvegarde du modèle entraîné
model_path = 'api/anomaly_model_scoring_credit.pkl'
joblib.dump(rf_model, model_path)
print(f"Modèle sauvegardé dans {model_path}")

Modèle sauvegardé dans api/anomaly_model_scoring_credit.pkl
